# Flagging days/periods with Snow and Heavy Rain



In [7]:
# import relevant modules
import configparser
from psycopg2 import connect
import psycopg2.sql as pg
import pandas.io.sql as pandasql
import datetime

# connect to database
CONFIG = configparser.ConfigParser()
CONFIG.read(r'C:\Users\alouis2\Documents\Python Scripts\db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

The following code automates the flagged data creation process through two functions. `formatter` takes in a dictionary who's keys are snow/heavy rain dates and who's values are time periods with snow corresponding to their key. This function returns a string that is used by the `flagged` function that returns a dataframe containing flagged data. 

In [8]:
december = {datetime.date(2017,12,11):['AM Peak', 'PM Peak', 'Evening'], 
                datetime.date(2017,12,12):['AM Peak', 'Midday'],
                datetime.date(2017,12,13):['AM Peak', 'Evening'],
                datetime.date(2017,12,15):['Midday', 'PM Peak'],
                datetime.date(2017,12,18):['AM Peak'],
                datetime.date(2017,12,21):['Evening'],
                datetime.date(2017,12,22):['AM Peak', 'Midday', 'PM Peak'],
                datetime.date(2017,12,29):['AM Peak', 'Midday']}

january = {datetime.date(2018,1,2):['AM Peak', 'Midday', 'Evening'],
          datetime.date(2018,1,7):['PM Peak'],
          datetime.date(2018,1,8):['AM Peak', 'Midday'],
          datetime.date(2018,1,12):['Midday', 'PM Peak', 'Evening'],
          datetime.date(2018,1,15):['Midday', 'PM Peak', 'Evening'],
          datetime.date(2018,1,16):['AM Peak', 'Evening'],
          datetime.date(2018,1,29):['Midday', 'PM Peak', 'Evening'],
          datetime.date(2018,1,30):['AM Peak'],
          datetime.date(2018,1,31):['AM Peak', 'Midday']}

february = {datetime.date(2018, 2, 3):['AM Peak', 'Midday', 'PM Peak', 'Evening'],
            datetime.date(2018, 2, 4):['Midday', 'PM Peak', 'Evening'],
            datetime.date(2018, 2, 7):['AM Peak', 'Midday', 'PM Peak'],
            datetime.date(2018, 2, 9):['AM Peak', 'Midday', 'PM Peak','Evening'],
            datetime.date(2018, 2, 10):['AM Peak', 'Midday', 'PM Peak', 'Evening'],
            datetime.date(2018, 2, 11):['Midday', 'PM Peak']}

march = {datetime.date(2018,3,1):['Evening'], 
          datetime.date(2018,3,7):['Midday'],
          datetime.date(2018,3,9):['Midday'],
          datetime.date(2018,3,12):['AM Peak', 'Midday', 'PM Peak'], 
          datetime.date(2018,3,15):['Midday', 'PM Peak'], 
          datetime.date(2018,3,29):['Midday', 'PM Peak', 'Evening']}

lof_months = [december, january, february, march]

In [9]:
def flagged(x):
    
    ## dict -> pd.Dataframe
    ## requires: dict must have format as specified above, i.e. the key must be a datetime
    ##           object and the value must be a list of time periods
       
    def formatter(x):
        string = ''' dt = '%s' '''
        for i in range(len(x)-1):
            string = string + '''or dt = '%s' '''
        string = string % tuple([key for key in x])
        return  '''select pilot.dt, pilot.street, pilot.direction, pilot.period, 
                    pilot.tt as pilot, baseline.tt as baseline 
                    from king_pilot.dash_daily_dev as pilot,  king_pilot.dash_baseline_dev as baseline 
                    where (%s)
                    and category = 'Pilot' and  pilot.day_type = 'Weekday' and pilot.street = baseline.street 
                    and pilot.direction = baseline.direction and pilot.day_type = baseline.day_type 
                    and pilot.period = baseline.period order by dt, street, direction, period''' % string
    string = formatter(x)
    sql = pg.SQL(string)
    snow = pandasql.read_sql(sql, con)
    change = [snow.pilot.values[i]-snow.baseline.values[i] for i in range(len(snow))]
    snow['change'] = change
    flag = [0 for i in range(len(snow))]
    snow['flag'] = flag 
    for key in x:
        for i in x[key]:
            for j in range(len(snow)):
                if snow.dt.values[j] == key and snow.period.values[j] == i:
                    snow.flag.values[j] += 1        
    return snow



The `return_snow` function accepts a list of dictionaries with snow periods, as indicated above. The function returns a dataframe with all snow flagged datetimes. 

In [10]:
def return_snow(lof_months):
    total = flagged(december)
    for i in lof_months[2:]:
        total = total.append(flagged(i), ignore_index = True)
    return total 

In [11]:
snowflags = return_snow(lof_months)

snowflags

,dt,street,direction,period,pilot,baseline,change,flag
0,2017-12-11,Adelaide,Eastbound,AM Peak,7.184190,7.864729,-0.680538,1
1,2017-12-11,Adelaide,Eastbound,Evening,6.146098,8.071130,-1.925032,1
2,2017-12-11,Adelaide,Eastbound,Midday,8.980245,9.410475,-0.430230,0
3,2017-12-11,Adelaide,Eastbound,PM Peak,25.102782,14.699972,10.402809,1
4,2017-12-11,Bathurst,Northbound,AM Peak,5.440064,5.298643,0.141421,1
5,2017-12-11,Bathurst,Northbound,Evening,4.239965,5.412162,-1.172197,1
6,2017-12-11,Bathurst,Northbound,Midday,5.070752,5.607554,-0.536801,0
7,2017-12-11,Bathurst,Northbound,PM Peak,6.383813,6.108102,0.275711,1
8,2017-12-11,Bathurst,Southbound,AM Peak,4.559895,5.109653,-0.549758,1
9,2017-12-11,Bathurst,Southbound,Evening,3.887059,5.248978,-1.361919,1
